In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
import pyomo.environ as pyo

from idaes.core.util.model_diagnostics import DegeneracyHunter

In [3]:
m = pyo.ConcreteModel()

m.I = pyo.Set(initialize=[i for i in range(5)])

m.x = pyo.Var(m.I, bounds=(-10, 10), initialize=1.0)

m.con1 = pyo.Constraint(expr=m.x[0] + m.x[1] - m.x[3] >= 10)
m.con2 = pyo.Constraint(expr=m.x[0] * m.x[3] + m.x[1] >= 0)
m.con3 = pyo.Constraint(expr=m.x[4] * m.x[3] + m.x[0] * m.x[3] - m.x[4] == 0)

m.obj = pyo.Objective(expr=sum(m.x[i] ** 2 for i in m.I))

m.pprint()

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   -10 :   1.0 :    10 : False : False :  Reals
          1 :   -10 :   1.0 :    10 : False : False :  Reals
          2 :   -10 :   1.0 :    10 : False : False :  Reals
          3 :   -10 :   1.0 :    10 : False : False :  Reals
          4 :   -10 :   1.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2

3 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper : Active
        None :  10.0 : x[0] + x[1] - x[3] :  +Inf :   True
    con2 : Size=1, Index=None, Active=

In [4]:
# Specify Ipopt as the solver
opt = pyo.SolverFactory("ipopt")

# Specifying an iteration limit of 0 allows us to inspect the initial point
opt.options["max_iter"] = 0

# "Solving" the model with an iteration limit of 0 load the initial point and applies
# any preprocessors (e.g., enforces bounds)
opt.solve(m, tee=True)

# Create Degeneracy Hunter object
# The Degeneracy Hunter algorithm needs a MILP solver
# Here we specify CBC, an open source solver
dh = DegeneracyHunter(m, solver=pyo.SolverFactory("cbc"))

Ipopt 3.13.2: max_iter=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. 

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.


In [5]:
dh.check_residuals(tol=0.1)

 
All constraints with residuals larger than 0.1 :
Count	Name	|residual|
0 	 con1 	 9.0
1 	 con3 	 1.0


dict_keys([<pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE500ACE0>, <pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE500B290>])

In [6]:
dh.check_variable_bounds(tol=1.0)

 
No variables within 1.0 (absolute) of their bounds.


In [7]:
opt.options["max_iter"] = 50
opt.solve(m, tee=True)

Ipopt 3.13.2: max_iter=50


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation.

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.08019137382507324}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
dh.check_residuals(tol=1e-14)

 
All constraints with residuals larger than 1e-14 :
Count	Name	|residual|
0 	 con1 	 9.97185747309004e-08
1 	 con2 	 7.942586144338293e-09
2 	 con3 	 2.2426505097428162e-14


dict_keys([<pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE500ACE0>, <pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE500B220>, <pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE500B290>])

In [9]:
dh.check_variable_bounds(tol=1e-5)

 
No variables within 1e-05 (absolute) of their bounds.


In [10]:
dh.check_rank_equality_constraints()


Checking rank of Jacobian of equality constraints...
Model contains 1 equality constraints and 5 variables.
Only singular value: 2.23606797749979


0

In [11]:
def example2(with_degenerate_constraint=True):
    """Create the Pyomo model for Example 2

    Arguments:
        with_degenerate_constraint: Boolean, if True, include the redundant linear constraint

    Returns:
        m2: Pyomo model
    """

    m2 = pyo.ConcreteModel()

    m2.I = pyo.Set(initialize=[i for i in range(1, 4)])

    m2.x = pyo.Var(m2.I, bounds=(0, 5), initialize=1.0)

    m2.con1 = pyo.Constraint(expr=m2.x[1] + m2.x[2] >= 1)
    m2.con2 = pyo.Constraint(expr=m2.x[1] + m2.x[2] + m2.x[3] == 1)
    m2.con3 = pyo.Constraint(expr=m2.x[2] - 2 * m2.x[3] <= 1)
    m2.con4 = pyo.Constraint(expr=m2.x[1] + m2.x[3] >= 1)

    if with_degenerate_constraint:
        m2.con5 = pyo.Constraint(expr=m2.x[1] + m2.x[2] + m2.x[3] == 1)

    m2.obj = pyo.Objective(expr=sum(m2.x[i] for i in m2.I))

    m2.pprint()

    return m2


# Create the Pyomo model for Example 2 including the redundant constraint
m2 = example2()

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    x : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     5 : False : False :  Reals
          2 :     0 :   1.0 :     5 : False : False :  Reals
          3 :     0 :   1.0 :     5 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3]

5 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :   1.0 : x[1] + x[2] :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper : Active
        None :   1.0 : x[1] + x[2] + x[3] :   1.0 :   True
    con3 : Size=1, Index=None, Active=True
     

In [12]:
# Specifying an iteration limit of 0 allows us to inspect the initial point
opt.options["max_iter"] = 0

# "Solving" the model with an iteration limit of 0 load the initial point and applies
# any preprocessors (e.g., enforces bounds)
opt.solve(m2, tee=True)

# Create Degeneracy Hunter object
# The Degeneracy Hunter algorithm needs a MILP solver
# Here we specify CBC, an open source solver
dh2 = DegeneracyHunter(m2, solver=pyo.SolverFactory("cbc"))

Ipopt 3.13.2: max_iter=0


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. 

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.


In [13]:
dh2.check_residuals(tol=0.1)

 
All constraints with residuals larger than 0.1 :
Count	Name	|residual|
0 	 con2 	 2.0
1 	 con5 	 2.0


dict_keys([<pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE508EC00>, <pyomo.core.base.constraint.ScalarConstraint object at 0x0000017CE508ED50>])

In [14]:
opt.options["max_iter"] = 50
opt.solve(m2, tee=True)

for i in m2.I:
    print("x[", i, "]=", m2.x[i]())

Ipopt 3.13.2: max_iter=50


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation.

x[ 1 ]= 1.0000000099975996
x[ 2 ]= 0.0
x[ 3 ]= 0.0


In [15]:
n_deficient = dh2.check_rank_equality_constraints()


Checking rank of Jacobian of equality constraints...
Model contains 2 equality constraints and 3 variables.
Computing the 1 smallest singular value(s)
Smallest singular value(s):
1.923E-16


In [16]:
ds2 = dh2.find_candidate_equations(verbose=True, tee=True)

*** Searching for a Single Degenerate Set ***
Building MILP model...
2 Set Declarations
    C : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}
    V : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

4 Var Declarations
    abs_nu : Size=2, Index=C
        Key : Lower : Value : Upper        : Fixed : Stale : Domain
          0 :     0 :  None : 100000.00001 : False :  True :  Reals
          1 :     0 :  None : 100000.00001 : False :  True :  Reals
    nu : Size=2, Index=C
        Key : Lower         : Value : Upper        : Fixed : Stale : Domain
          0 : -100000.00001 :   1.0 : 100000.00001 : False : False :  Reals
          1 : -100000.00001 :   1.0 : 100000.00001 : False : False :  Reals
    y_neg : Size=2, Index=C
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 

Welcome to the CBC MILP Solver 
Version: 2.10.4 
Build Date: Nov  2 2022 

command line - C:\Users\dang\AppData\Local\idaes\bin\cbc.exe -printingOptions all -import C:\Users\dang\AppData\Local\Temp\tmpcodglu4j.pyomo.lp -stat=1 -solve -solu C:\Users\dang\AppData\Local\Temp\tmpcodglu4j.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 9 (-8) rows, 7 (-2) columns and 20 (-15) elements
Statistics for presolved model
Original problem has 4 integers (4 of which binary)
Presolved problem has 4 integers (4 of which binary)
==== 5 zero objective 2 different
5 variables have objective of 0
2 variables have objective of 1
==== absolute objective values 2 different
5 variables have objective of 0
2 variables have objective of 1
==== for integers 4 zero objective 1 different
4 variables have objective of 0
==== for integers absolute objective values 1 different
4 variables have objective of 0
===== end objective counts


Problem has 9 rows, 7 columns (2 

In [17]:
ids = dh2.find_irreducible_degenerate_sets(verbose=True)

*** Searching for Irreducible Degenerate Sets ***
Building MILP model...
Solving MILP 1 of 2 ...


Solving MILP 2 of 2 ...



Irreducible Degenerate Set 0
nu	Constraint Name
1.0 	 con2
-1.0 	 con5

Irreducible Degenerate Set 1
nu	Constraint Name
-1.0 	 con2
1.0 	 con5


In [18]:
m2b = example2(with_degenerate_constraint=False)

1 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    x : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     5 : False : False :  Reals
          2 :     0 :   1.0 :     5 : False : False :  Reals
          3 :     0 :   1.0 :     5 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3]

4 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :   1.0 : x[1] + x[2] :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body               : Upper : Active
        None :   1.0 : x[1] + x[2] + x[3] :   1.0 :   True
    con3 : Size=1, Index=None, Active=True
     

In [19]:
opt.options["max_iter"] = 50
opt.solve(m2b, tee=True)

for i in m2b.I:
    print("x[", i, "]=", m.x[i]())

Ipopt 3.13.2: max_iter=50


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation.

x[ 1 ]= 5.061595738300216
x[ 2 ]= 9.184026563775058e-26
x[ 3 ]= -3.4867300513803765
